In [28]:
import numpy as np
import pandas as pd
import mlflow

In [ ]:
mlflow.set_tracking_uri('http://ec2-51-20-64-139.eu-north-1.compute.amazonaws.com:5000')
mlflow.set_experiment('exp 4 - Handling imbalance data')

<Experiment: artifact_location='s3://neeraj-first-bucket/309714134278689943', creation_time=1736860723048, experiment_id='309714134278689943', last_update_time=1736860723048, lifecycle_stage='active', name='Exp 4 - Handling imbalance data', tags={}>

In [30]:
!pip install imblearn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [32]:
df = pd.read_csv('dataset.csv')
df.head()
df.dropna(inplace=True)

In [33]:
def run_experiment(imbalance_method):
    ngram_range = (1,2)
    max_features = 2000

    # split data
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2)

    # vectorizer
    vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
    X_train_trf = vectorizer.fit_transform(X_train)
    X_test_trf = vectorizer.transform(X_test)

    # class imbalance
    if  imbalance_method == 'class_weights':
        class_weight = 'balanced'
    else:
        class_weight = None
        # Resampleing tech
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_trf, y_train = smote.fit_resample(X_train_trf, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_trf, y_train = adasyn.fit_resample(X_train_trf, y_train)
        elif imbalance_method == 'undersampling':
            random_sample = RandomUnderSampler(random_state=42)
            X_train_trf, y_train = random_sample.fit_resample(X_train_trf, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_trf, y_train = smote_enn.fit_resample(X_train_trf, y_train)
    
    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForestClassifier_Tfidf_bigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", f"RandomForestClassifier")
        mlflow.set_tag("description", f"RandomForest with Tf-Idf bigrams, imbalance_handling_method={imbalance_method}")

        mlflow.log_param("class_weight", f"{class_weight}")
        mlflow.log_param("imbalance_method", f"{imbalance_method}")
        mlflow.log_param("vectorizer_type", f"Tf-idf")
        mlflow.log_param("ngram_range", f"{ngram_range}")
        mlflow.log_param("vectorizer_max_features", f"{max_features}")

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
        model.fit(X_train_trf, y_train)
        y_pred = model.predict(X_test_trf)

        # metrics
        accuracy = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)
        conf_matrix = confusion_matrix(y_test, y_pred)

        mlflow.log_metric("accuracy", accuracy)
        for label, metrics in class_report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        


        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix: TF-IDF Bigrams, Imbalance={imbalance_method}")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.savefig(f"confusion_matrix_{imbalance_method}.png")
        mlflow.log_artifact(f"confusion_matrix_{imbalance_method}.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

In [34]:
# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_experiment(method)

2025/01/14 19:01:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForestClassifier_Tfidf_bigrams at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943/runs/64436155b95345de9ca6a0bdb28d47c4
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943


2025/01/14 19:02:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForestClassifier_Tfidf_bigrams at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943/runs/7a35d5ba1ce442d099fd73f5d49e7f53
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943


2025/01/14 19:03:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForestClassifier_Tfidf_bigrams at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943/runs/824bed9cb5f4445eb9d68fc03520de08
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943


2025/01/14 19:03:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForestClassifier_Tfidf_bigrams at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943/runs/c3a9ce7f052c4fc4911c04f3e0947566
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943


2025/01/14 19:05:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForestClassifier_Tfidf_bigrams at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943/runs/2394a27d7acc4db8a41b18a61d283202
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/309714134278689943
